<a href="https://colab.research.google.com/github/5dongdong/PY_AI/blob/master/XGBoost%EC%98%B5%ED%8A%9C%EB%82%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [ ]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Medical_NoShow.csv')  

In [ ]:
df

,Age,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,No-show,Reservation_wait_days,Total_cancellations,Handicap_0,Handicap_1,Handicap_2,Handicap_3,Handicap_4,Gender_F,Gender_M
0,62,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
1,56,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,62,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,8,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,56,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110514,56,0,0,0,0,1,0,35,1,1,0,0,0,0,1,0
110515,51,0,0,0,0,1,0,35,1,1,0,0,0,0,1,0
110516,21,0,0,0,0,1,0,41,0,1,0,0,0,0,1,0
110517,38,0,0,0,0,1,0,41,0,1,0,0,0,0,1,0


In [ ]:
X = df[['Gender_F','Gender_M', 'Diabetes','Hipertension', 'Scholarship', 'SMS_received',
        'Handicap_0','Handicap_1','Handicap_2','Handicap_3','Handicap_4', 'Total_cancellations', 'Age', 'Reservation_wait_days']]
y = df["No-show"] 

x_train, x_test, y_train,y_test = train_test_split(
    X, y, train_size=0.8, shuffle=True, random_state=42
)
     

In [ ]:
# 변수 설정
n_splits = 11    # 보통 홀수로 들어감
random_state = 42
scaler = MinMaxScaler()

In [ ]:
kfold = KFold(n_splits=n_splits, shuffle=True, 
              random_state=random_state)


# Scaler
scaler.fit(x_train)                 
x = scaler.transform(x_train)   # train 은 fit, transform 모두 해줘야 함
x = scaler.transform(x_test) 

In [ ]:
def objectiveXGB(trial: Trial, x_train, y_train, x_test, y_test):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 4000),
        'max_depth': trial.suggest_int('depth', 1, 16),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 1),
        'random_state': trial.suggest_int('random_state', 1, 2000)
    }
    # 학습 모델 생성
    model = XGBClassifier(**param)
    XGB_model = model.fit(x_train, y_train, verbose=True) # 학습 진행
    # 모델 성능 확인
    y_pred = XGB_model.predict(x_test)
    score = accuracy_score(y_pred, y_test)
    return score


# MAE가 최대가 되는 방향으로 학습을 진행
# TPESampler: TPE(Tree-structured Parzen Estimator) 알고리즘을 사용하는 샘플러
study = optuna.create_study(direction='maximize', sampler=TPESampler())

# n_trials를 지정해주지 않으면 무한 반복
study.optimize(lambda trial: objectiveXGB(trial, x_train, y_train, x_test, y_test), n_trials=5)

# 최적의 trial 결과 출력
print('Best trial - Score: {}, Params: {}'.format(study.best_trial.value, study.best_trial.params))

# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study).show()

# 하이퍼파라미터 최적화 과정을 확인하는 그래프
optuna.visualization.plot_optimization_history(study).show()

[I 2023-05-18 07:15:46,913] A new study created in memory with name: no-name-2b796253-53c1-4ab1-8c94-7431a0380c53
[I 2023-05-18 07:18:47,277] Trial 0 finished with value: 0.9234527687296417 and parameters: {'n_estimators': 2464, 'depth': 5, 'learning_rate': 0.050648587564364585, 'random_state': 815}. Best is trial 0 with value: 0.9234527687296417.
[I 2023-05-18 07:28:02,409] Trial 1 finished with value: 0.9047683677162505 and parameters: {'n_estimators': 2529, 'depth': 14, 'learning_rate': 0.4620802469868892, 'random_state': 558}. Best is trial 0 with value: 0.9234527687296417.
[I 2023-05-18 07:30:19,111] Trial 2 finished with value: 0.9068041983351429 and parameters: {'n_estimators': 1301, 'depth': 7, 'learning_rate': 0.8724229229596902, 'random_state': 1771}. Best is trial 0 with value: 0.9234527687296417.
[I 2023-05-18 07:31:01,739] Trial 3 finished with value: 0.923362287368802 and parameters: {'n_estimators': 673, 'depth': 4, 'learning_rate': 0.2898593173447003, 'random_state': 16

Best trial - Score: 0.9234527687296417, Params: {'n_estimators': 2464, 'depth': 5, 'learning_rate': 0.050648587564364585, 'random_state': 815}


In [ ]:
# 2. 모델
model = XGBClassifier()

# 3. 훈련
model.fit(x_train, y_train)

#4. 평가, 예측
result = model.score(x_test,y_test)
print('결과 acc : ', result)
#print('결과 r2 : ', result)

score = cross_val_score( model, 
                        x_train, y_train,
                        cv = kfold )  #cv = cross validation_

y_predict = cross_val_predict(model,
                              x_test, y_test,
                              cv = kfold)

acc = accuracy_score(y_test, y_predict)
print('cv pred acc : ', acc )

결과 acc :  0.9205573651827723
cv pred acc :  0.9153999276149113
